In [ ]:
#In this file the assertion of the paper, that the microstructure
#does not impede transmission is put to the test

In [ ]:
#Calculating the critical angle for a air glass interface with n_glass = 1.5

In [2]:
import numpy as np
theta_c = np.arcsin(1/1.5)*180/np.pi
print(theta_c)

41.810314895778596


In [ ]:
#First for straight incidence

In [ ]:
#For the microdome geometry

In [ ]:
#First without domes

In [79]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi, arctan
from raysect.optical.library import schott
import numpy as np

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D #Pixel comes from: source/raysect/optical/observer/nonimaging

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials
perfect_reflector = mat_materials.perfect_mirror()
foil_width = 400
foil_thickness = 75
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8


# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**8  #10**7 samples takes about 30 seconds
alpha = 1 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source) #This equation is based on the image on page 32 


source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") 
foil_material = mat_materials.constant_n_glass(1.5)


foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
           parent=world,material=foil_material)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform= rotate_x(-90) * translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))


Render time: 1.000s (0.08% complete, 242.8k rays)
Render time: 2.005s (0.22% complete, 433.7k rays)
Render time: 3.006s (0.39% complete, 547.6k rays)
Render time: 4.007s (0.66% complete, 842.6k rays)
Render time: 5.008s (0.89% complete, 708.5k rays)
Render time: 6.014s (1.09% complete, 623.6k rays)
Render time: 7.036s (1.29% complete, 639.9k rays)
Render time: 8.056s (1.46% complete, 522.2k rays)
Render time: 9.063s (1.77% complete, 961.0k rays)
Render time: 10.066s (2.11% complete, 1077.6k rays)
Render time: 11.076s (2.46% complete, 1077.3k rays)
Render time: 12.081s (2.80% complete, 1078.1k rays)
Render time: 13.084s (3.15% complete, 1090.8k rays)
Render time: 14.112s (3.49% complete, 1050.5k rays)
Render time: 15.142s (3.78% complete, 910.2k rays)
Render time: 16.158s (4.03% complete, 790.6k rays)
Render time: 17.163s (4.36% complete, 1031.4k rays)
Render time: 18.178s (4.71% complete, 1099.8k rays)
Render time: 19.200s (5.07% complete, 1099.2k rays)
Render time: 20.207s (5.42% comp

In [36]:
#With domes

In [78]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi
from raysect.optical.library import schott

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials #To use the self defined materials
from mat_functions import Microarrays

perfect_reflector = mat_materials.perfect_mirror()
foil_width = 400
foil_thickness = 75
height = 65
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8

# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**8  #10**6 samples takes about 30 seconds
alpha = 1 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
#See page 32 notebook
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source)
#print('d_source',d_source)
source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") #mat_materials.constant_n_glass(1.5) 
foil_material = mat_materials.constant_n_glass(1.5)


#foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
#           parent=world,material=foil_material)

foil = Microarrays.micro_dome_array(foil_width, height, foil_thickness, 1, foil_material, world)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform=rotate_x(-90) *  translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

Render time: 1.017s (0.02% complete, 100.9k rays)
Render time: 2.161s (0.04% complete, 114.9k rays)
Render time: 3.177s (0.08% complete, 228.7k rays)
Render time: 4.181s (0.10% complete, 172.0k rays)
Render time: 5.188s (0.14% complete, 224.3k rays)
Render time: 6.190s (0.18% complete, 231.5k rays)
Render time: 7.194s (0.23% complete, 291.9k rays)
Render time: 8.205s (0.29% complete, 345.0k rays)
Render time: 9.207s (0.35% complete, 326.9k rays)
Render time: 10.217s (0.40% complete, 303.6k rays)
Render time: 11.223s (0.44% complete, 269.3k rays)
Render time: 12.236s (0.50% complete, 347.9k rays)
Render time: 13.602s (0.51% complete, 67.8k rays)
Render time: 14.602s (0.56% complete, 294.9k rays)
Render time: 15.608s (0.61% complete, 270.3k rays)
Render time: 16.618s (0.66% complete, 300.7k rays)
Render time: 17.620s (0.71% complete, 273.0k rays)
Render time: 18.625s (0.76% complete, 321.3k rays)
Render time: 19.639s (0.82% complete, 337.6k rays)
Render time: 20.657s (0.88% complete, 340

In [ ]:
#Prisms

In [62]:
#From the paper: w = 300 & α=25, thus:
from numpy import tan, pi
h_prism = round(150*tan(25/180*pi),2)
print(h_prism)

69.95


In [ ]:
#Structure absent (the foil width here is diferent than for the microdomes, since the prism foil has different measurements)

In [63]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi, arctan
from raysect.optical.library import schott

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D #Pixel comes from: source/raysect/optical/observer/nonimaging

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials
perfect_reflector = mat_materials.perfect_mirror()
foil_width = 300
foil_thickness = 75
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8


# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**7  #10**7 samples takes about 30 seconds
alpha = 2 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source) #This equation is based on the image on page 32 


source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") 
foil_material = mat_materials.constant_n_glass(1.5)


foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
           parent=world,material=foil_material)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform= rotate_x(-90) * translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

Render time: 1.001s (0.78% complete, 247.6k rays)
Render time: 2.009s (1.78% complete, 315.5k rays)
Render time: 3.016s (4.38% complete, 822.9k rays)
Render time: 4.017s (7.42% complete, 962.0k rays)
Render time: 5.022s (10.44% complete, 956.0k rays)
Render time: 6.023s (12.51% complete, 654.7k rays)
Render time: 7.036s (15.81% complete, 1045.1k rays)
Render time: 8.038s (19.08% complete, 1036.5k rays)
Render time: 9.043s (22.16% complete, 973.1k rays)
Render time: 10.399s (23.41% complete, 395.9k rays)
Render time: 11.402s (26.70% complete, 1040.0k rays)
Render time: 12.410s (30.05% complete, 1062.4k rays)
Render time: 13.419s (32.94% complete, 914.7k rays)
Render time: 14.421s (35.72% complete, 879.3k rays)
Render time: 15.495s (37.21% complete, 473.3k rays)
Render time: 16.503s (38.71% complete, 473.2k rays)
Render time: 17.509s (39.75% complete, 327.8k rays)
Render time: 18.510s (40.88% complete, 360.3k rays)
Render time: 19.510s (43.64% complete, 873.1k rays)
Render time: 20.546s 

In [ ]:
#Structure present

In [64]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi
from raysect.optical.library import schott
import numpy as np

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials #To use the self defined materials
from mat_functions import Microarrays

perfect_reflector = mat_materials.perfect_mirror()
foil_width = 300
foil_thickness = 75
height = 69.95
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8

# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**7  #10**6 samples takes about 30 seconds
alpha = 2 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
#See page 32 notebook
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source)
#print('d_source',d_source)
source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") #mat_materials.constant_n_glass(1.5) 
foil_material = mat_materials.constant_n_glass(1.5)


#foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
#           parent=world,material=foil_material)

foil = Microarrays.prism_array(foil_width, height, foil_thickness, 1, foil_material, world)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform=rotate_x(-90) *  translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

Render time: 1.011s (0.08% complete, 110.5k rays)
Render time: 2.046s (0.29% complete, 290.0k rays)
Render time: 3.090s (0.37% complete, 106.7k rays)
Render time: 4.106s (0.56% complete, 283.1k rays)
Render time: 5.112s (0.81% complete, 339.7k rays)
Render time: 6.117s (1.02% complete, 283.0k rays)
Render time: 7.120s (1.26% complete, 348.8k rays)
Render time: 8.124s (1.52% complete, 347.2k rays)
Render time: 9.138s (1.77% complete, 349.3k rays)
Render time: 10.153s (2.03% complete, 351.1k rays)
Render time: 11.153s (2.27% complete, 340.7k rays)
Render time: 12.154s (2.52% complete, 354.6k rays)
Render time: 13.158s (2.77% complete, 346.0k rays)
Render time: 14.160s (2.93% complete, 211.2k rays)
Render time: 15.165s (3.19% complete, 351.9k rays)
Render time: 16.172s (3.36% complete, 248.6k rays)
Render time: 17.179s (3.62% complete, 352.8k rays)
Render time: 18.190s (3.87% complete, 349.8k rays)
Render time: 19.195s (4.12% complete, 350.7k rays)
Render time: 20.203s (4.38% complete, 35

In [ ]:
#Pyramids

In [71]:
#pyramid width = 10
#angle = 25
pyramid_height = h_prism = round(5*tan(25/180*pi),2)
print(pyramid_height)

2.33


In [ ]:
#Structure absent

In [ ]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi, arctan
from raysect.optical.library import schott

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D #Pixel comes from: source/raysect/optical/observer/nonimaging

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials
perfect_reflector = mat_materials.perfect_mirror()
foil_width = 10
foil_thickness = 75
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8


# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**7  #10**7 samples takes about 30 seconds
alpha = 2 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source) #This equation is based on the image on page 32 


source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") 
foil_material = mat_materials.constant_n_glass(1.5)


foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
           parent=world,material=foil_material)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform= rotate_x(-90) * translate(0, 0, -foil_thickness-0.01)  # Position the observer
    ,quiet=True)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

In [73]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi
from raysect.optical.library import schott
import numpy as np

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials #To use the self defined materials
from mat_functions import Microarrays

perfect_reflector = mat_materials.perfect_mirror()
foil_width = 10
foil_thickness = 75
height = 2.33
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8

# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**7  #10**6 samples takes about 30 seconds
alpha = 2 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
#See page 32 notebook
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source)
#print('d_source',d_source)
source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") #mat_materials.constant_n_glass(1.5) 
foil_material = mat_materials.constant_n_glass(1.5)


#foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
#           parent=world,material=foil_material)

foil = Microarrays.pyramid_array(foil_width, height, foil_thickness, 1,  world, foil_material)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform=rotate_x(-90) *  translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

Render time: 1.005s (0.15% complete, 212.3k rays)
Render time: 2.027s (0.35% complete, 286.7k rays)
Render time: 3.029s (0.59% complete, 323.7k rays)
Render time: 4.040s (0.76% complete, 240.6k rays)
Render time: 5.057s (0.83% complete, 107.6k rays)
Render time: 6.107s (0.92% complete, 117.7k rays)
Render time: 7.155s (1.01% complete, 130.9k rays)
Render time: 8.174s (1.13% complete, 171.3k rays)
Render time: 9.190s (1.31% complete, 252.8k rays)
Render time: 10.197s (1.54% complete, 322.2k rays)
Render time: 11.198s (1.78% complete, 327.1k rays)
Render time: 12.200s (2.01% complete, 321.2k rays)
Render time: 13.201s (2.24% complete, 321.9k rays)
Render time: 14.205s (2.44% complete, 289.3k rays)
Render time: 15.228s (2.68% complete, 327.6k rays)
Render time: 16.238s (2.88% complete, 267.4k rays)
Render time: 17.239s (3.10% complete, 320.8k rays)
Render time: 18.241s (3.33% complete, 320.6k rays)
Render time: 19.248s (3.56% complete, 318.1k rays)
Render time: 20.396s (3.59% complete, 42

In [ ]:
#Cone

In [ ]:
#Absent

In [75]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi, arctan
from raysect.optical.library import schott

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D #Pixel comes from: source/raysect/optical/observer/nonimaging

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials
perfect_reflector = mat_materials.perfect_mirror()
foil_width = 250
foil_thickness = 75
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8


# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**7  #10**7 samples takes about 30 seconds
alpha = 2 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source) #This equation is based on the image on page 32 


source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") 
foil_material = mat_materials.constant_n_glass(1.5)


foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
           parent=world,material=foil_material)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform= rotate_x(-90) * translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

Render time: 1.002s (0.96% complete, 306.9k rays)
Render time: 2.016s (1.93% complete, 311.3k rays)
Render time: 3.017s (3.54% complete, 514.8k rays)
Render time: 4.029s (4.81% complete, 407.0k rays)
Render time: 5.030s (6.30% complete, 473.6k rays)
Render time: 6.044s (9.10% complete, 895.6k rays)
Render time: 7.047s (12.24% complete, 1006.4k rays)
Render time: 8.047s (15.40% complete, 1010.2k rays)
Render time: 9.050s (18.40% complete, 959.7k rays)
Render time: 10.053s (20.09% complete, 541.1k rays)
Render time: 11.058s (21.98% complete, 604.6k rays)
Render time: 12.066s (25.22% complete, 1035.3k rays)
Render time: 13.079s (28.48% complete, 1043.0k rays)
Render time: 14.082s (31.76% complete, 1048.4k rays)
Render time: 15.106s (35.04% complete, 1049.2k rays)
Render time: 16.170s (37.30% complete, 722.8k rays)
Render time: 17.182s (40.10% complete, 895.9k rays)
Render time: 18.307s (40.80% complete, 224.1k rays)
Render time: 19.314s (43.95% complete, 1008.3k rays)
Render time: 20.327s

In [ ]:
#Present

In [4]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi
from raysect.optical.library import schott
import numpy as np

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials #To use the self defined materials
from mat_functions import Microarrays

perfect_reflector = mat_materials.perfect_mirror()
foil_width = 250
foil_thickness = 75
height = 115
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8

# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**7  #10**6 samples takes about 30 seconds
alpha = 2 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
#See page 32 notebook
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source)
#print('d_source',d_source)
source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") #mat_materials.constant_n_glass(1.5) 
foil_material = mat_materials.constant_n_glass(1.5)


#foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
#           parent=world,material=foil_material)

foil = Microarrays.cone_array(foil_width, height, foil_thickness, 1, foil_material,world)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform=rotate_x(-90) *  translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

Render time: 1.009s (0.06% complete, 32.3k rays)
Render time: 2.012s (0.17% complete, 62.9k rays)
Render time: 3.022s (0.34% complete, 92.3k rays)
Render time: 4.028s (0.47% complete, 74.9k rays)
Render time: 5.054s (0.61% complete, 78.8k rays)
Render time: 6.063s (0.89% complete, 158.0k rays)
Render time: 7.069s (1.27% complete, 211.6k rays)
Render time: 8.086s (1.62% complete, 196.0k rays)
Render time: 9.100s (2.02% complete, 224.8k rays)
Render time: 10.114s (2.36% complete, 184.1k rays)
Render time: 11.117s (2.57% complete, 121.7k rays)
Render time: 12.125s (2.79% complete, 115.4k rays)
Render time: 13.127s (3.08% complete, 164.6k rays)
Render time: 14.130s (3.43% complete, 194.2k rays)
Render time: 15.793s (3.60% complete, 94.8k rays)
Render time: 16.805s (4.00% complete, 221.8k rays)
Render time: 17.807s (4.36% complete, 202.0k rays)
Render time: 18.813s (4.64% complete, 155.1k rays)
Render time: 19.822s (5.03% complete, 212.8k rays)
Render time: 20.827s (5.40% complete, 204.8k r

In [ ]:
#Testing flat surface, prism and pyramids for a width of 600 and a height of 2*69.95 = 139.9

In [6]:
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi, arctan
from raysect.optical.library import schott
import numpy as np

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D #Pixel comes from: source/raysect/optical/observer/nonimaging

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials
perfect_reflector = mat_materials.perfect_mirror()
foil_width = 600
foil_thickness = 75
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8


# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**7  #10**7 samples takes about 30 seconds
alpha = 2 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source) #This equation is based on the image on page 32 


source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") 
foil_material = mat_materials.constant_n_glass(1.5)


foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
           parent=world,material=foil_material)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform= rotate_x(-90) * translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

Render time: 1.001s (0.31% complete, 94.7k rays)
Render time: 2.008s (0.88% complete, 176.6k rays)
Render time: 3.009s (1.79% complete, 280.8k rays)
Render time: 4.009s (2.48% complete, 213.1k rays)
Render time: 5.012s (3.77% complete, 396.3k rays)
Render time: 6.021s (4.97% complete, 371.5k rays)
Render time: 7.024s (5.93% complete, 296.1k rays)
Render time: 8.032s (6.67% complete, 228.1k rays)
Render time: 9.055s (7.29% complete, 190.2k rays)
Render time: 10.081s (7.76% complete, 146.5k rays)
Render time: 11.092s (8.77% complete, 311.9k rays)
Render time: 12.119s (9.83% complete, 324.0k rays)
Render time: 13.139s (11.33% complete, 462.0k rays)
Render time: 14.151s (12.66% complete, 410.5k rays)
Render time: 15.152s (13.95% complete, 399.3k rays)
Render time: 16.165s (15.46% complete, 464.1k rays)
Render time: 17.169s (16.95% complete, 461.3k rays)
Render time: 19.693s (18.25% complete, 400.6k rays)
Render time: 20.727s (19.70% complete, 447.0k rays)
Render time: 21.742s (21.18% compl

In [ ]:
#Test scaled up Pyramid
#Imports
#Width = 600, height = 139.9

In [7]:
#Test scaled up Pyramid
#Imports
import matplotlib.pyplot as plt
from raysect.optical import World, translate, rotate, Point3D, d65_white, ConstantSF
from raysect.optical.observer import PinholeCamera
from raysect.optical.material import UniformSurfaceEmitter, Lambert
from raysect.primitive import Sphere, Box, Cylinder, Intersect, Union, Subtract
from raysect.core import rotate_x, rotate_y, rotate_z
from numpy import tan, pi
from raysect.optical.library import schott
import numpy as np

#Imports for power measurement
from raysect.optical.observer import Pixel, PowerPipeline0D

#import from mat_functions
import import_ipynb                  #Function required to do imports from jupyter notebook files 
from mat_functions import mat_materials #To use the self defined materials
from mat_functions import Microarrays

perfect_reflector = mat_materials.perfect_mirror()
from raysect.optical import ConstantSF     

foil_width = 600
foil_thickness = 75
height = 2*69.95
r_source = 6.9634*10**8   #Radius of the light source (made significantly bigger than the foil to simulate reality)

#Radius and distance to the sun, to calculate d_source
distance_sun = 1.4976*10**11
r_sun = 6.9634*10**8

# Number of samples to be used in the observation. A larger number of samples increases accuracy.
samples = 10**7  #10**6 samples takes about 30 seconds
alpha = 2 #arctan(r_sun/distance_sun) #See page 32 notebook


world = World()

#Light source
#Calculating the distance to the source. 
#See page 32 notebook
d_source = np.tan((90-alpha)*np.pi/180)*(foil_width/2**0.5+r_source)
#print('d_source',d_source)
source_material = UniformSurfaceEmitter(d65_white, 1)
light_source = Sphere(r_source, transform=translate(0, d_source, 0),parent = world,material = source_material)

#This light source is used while testing stuff (to be removed later)
#test_source = light_source = Sphere(100, transform=translate(0, 1000, 0),
#                                parent = world,material = source_material) 


# Creating the foil
#foil_material = schott("N-BK7") #mat_materials.constant_n_glass(1.5) 
foil_material = mat_materials.constant_n_glass(1.5)


#foil = Box(Point3D(-foil_width/2,-foil_thickness,-foil_width/2),Point3D(foil_width/2,0,foil_width/2),
#           parent=world,material=foil_material)

foil = Microarrays.pyramid_array(foil_width, height, foil_thickness, 1,  world, foil_material)

#Adding the mirror edges
mirror_1 = Box(Point3D(-foil_width/2 - 1, -foil_thickness, -foil_width/2),
                  Point3D(-foil_width/2, 0, foil_width/2),
                  parent=world, material = perfect_reflector)
mirror_2 = Box(Point3D(foil_width/2, -foil_thickness, -foil_width/2),
                   Point3D(foil_width/2 + 1, 0, foil_width/2),
                   parent=world, material = perfect_reflector)
mirror_3 = Box(Point3D(-foil_width/2, -foil_thickness, foil_width/2),
                   Point3D(foil_width/2, 0, foil_width/2 + 1),
                   parent=world, material = perfect_reflector)
mirror_4 = Box(Point3D(-foil_width/2, -foil_thickness, -foil_width/2 - 1),
                  Point3D(foil_width/2, 0, -foil_width/2),
                  parent=world, material = perfect_reflector)


'''
#Placing a camera (this camera will be changed into a surface later)
# Camera setup
camera = PinholeCamera((512, 512),
        parent=world, transform= rotate_x(-90))

#Moving the camera down
camera.transform = camera.transform* translate(0, 0, -500)

camera.spectral_bins = 15
camera.pixel_samples = 100
camera.quiet = True
camera.ray_max_depth = 10
camera.ray_extinction_min_depth == 1
camera.ray_extinction_prob = 0.01

# Start ray tracing
plt.ion()
for p in range(1):
    print("Rendering pass {}...".format(p))
    camera.observe()
    print()

# Display final result
plt.ioff()
#camera.display()
#show()

print('done')
'''

#Now placing the observing plane instead of the camera


power = PowerPipeline0D(accumulate=False)
observing_plane = Pixel(
    [power], 
    x_width = foil_width, y_width = foil_width,  # Size of the observing plane
    #min_wavelength=min_wl, max_wavelength=max_wl,  # Wavelength range to observe
    spectral_bins=1,  # Number of wavelength bins
    pixel_samples=samples,  # Number of samples to use
    parent=world, 
    transform=rotate_x(-90) *  translate(0, 0, -foil_thickness-0.01)  # Position the observer
)
#Perform observation
observing_plane.observe()
measured_power = round(power.value.mean)
measured_error = round(power.value.error())
print('Measured power => {} +/- {} W'.format(measured_power, measured_error))

Render time: 1.027s (0.19% complete, 105.0k rays)
Render time: 2.037s (0.41% complete, 119.2k rays)
Render time: 3.042s (0.61% complete, 111.9k rays)
Render time: 4.054s (0.80% complete, 101.9k rays)
Render time: 5.078s (0.88% complete, 41.8k rays)
Render time: 6.086s (1.04% complete, 92.0k rays)
Render time: 7.095s (1.20% complete, 88.0k rays)
Render time: 8.108s (1.34% complete, 76.8k rays)
Render time: 9.121s (1.45% complete, 58.4k rays)
Render time: 10.158s (1.59% complete, 76.1k rays)
Render time: 11.210s (1.71% complete, 68.8k rays)
Render time: 12.219s (1.86% complete, 82.0k rays)
Render time: 13.220s (2.05% complete, 104.1k rays)
Render time: 14.226s (2.26% complete, 115.9k rays)
Render time: 15.270s (2.43% complete, 91.1k rays)
Render time: 16.277s (2.60% complete, 97.1k rays)
Render time: 17.304s (2.80% complete, 104.8k rays)
Render time: 18.314s (3.00% complete, 110.7k rays)
Render time: 19.318s (3.22% complete, 121.7k rays)
Render time: 20.325s (3.45% complete, 125.6k rays)